# Algoritmus SIMPLE pro Navierovy-Stokesovy rovnice

In [1]:
using PyPlot;

In [2]:
include("mesh.jl");
include("fields.jl");
include("operators.jl");

In [3]:
function div(ϕ::Array{Float64,1} , U::Field)
    A = spzeros(length(U.values),length(U.values))
    b = zeros(U.values)
    
    msh = U.mesh
    
    for f ∈ msh.faces
        o = f.owner
        n = f.neigh
    
        α = max(ϕ[f.id], 0.0)
        β = min(ϕ[f.id], 0.0)
        
        A[o,o] += α / msh.cells[o].vol
        A[o,n] += β / msh.cells[o].vol
            
        A[n,o] -= α / msh.cells[n].vol
        A[n,n] -= β / msh.cells[n].vol
    end

    for (name,faces) ∈ msh.patches
        bc = U.boundaries[name]
        for (i,f) ∈ enumerate(bc.faces)
            o = f.owner
            coeff = boundary_coeffs(bc, i)  # tj. ub = coef[1] + coef[2]*uin
            α = max(ϕ[f.id], 0.0)
            β = min(ϕ[f.id], 0.0)

            c = boundary_coeffs(bc, i)  # tj. ub = c[1] + c[2]*uin
            A[o,o] += (α + β*c[2]) / msh.cells[o].vol
            b[o]   += β*c[1] / msh.cells[o].vol
        end
    end

    
    return Equation(A, U.values, b)
end     

div (generic function with 1 method)

In [4]:
function create_fields(msh)
    U = VectorField(msh)
    set_dirichlet_patch!(U, "left", Vec2d(0,0));
    set_dirichlet_patch!(U, "right", Vec2d(0,0));
    set_dirichlet_patch!(U, "bottom", Vec2d(0,0));
    set_dirichlet_patch!(U, "top", Vec2d(1,0));

    p = ScalarField(msh);
    for name ∈ ["left", "right", "bottom", "top"]
        set_neumann_patch!(p, name, 0.0)
    end

    return (U,p)
end;

In [5]:
function interpolate_U_to_faces(U::VectorField)
    mesh = U.mesh
    nfaces = length(mesh.faces) + reduce(+, [length(faces) for (_,faces) ∈ mesh.patches])
    ϕ = zeros(nfaces)
    for f ∈ mesh.faces
        o = f.owner
        n = f.neigh
        ϕ[f.id] = dot(f.s, (U[o]+U[n])/2.)
    end

    for (name,faces) ∈ mesh.patches
        bc = U.boundaries[name]
        for (i,f) ∈ enumerate(bc.faces)
            o = f.owner
            Ub = boundary_value(bc, i)
            ϕ[f.id] = dot(f.s, Ub)
        end
    end

    return ϕ
end

interpolate_U_to_faces (generic function with 1 method)

In [6]:
function calculate_ϕ!(ϕ::Array{Float64,1}, Ubar::VectorField, p::ScalarField, ra::Array{Float64,1})
    mesh = Ubar.mesh
    nfaces = length(mesh.faces) + reduce(+, [length(faces) for (_,faces) ∈ mesh.patches])
    for f ∈ mesh.faces
        o = f.owner
        n = f.neigh
        δ = norm(mesh.cells[n].x - mesh.cells[o].x)
        pn = (p[n] - p[o]) / δ
        ϕ[f.id] = dot(f.s, (Ubar[o]+Ubar[n])/2.) - (ra[o]+ra[n])/2 * pn * norm(f.s)
    end

    for (name,faces) ∈ mesh.patches
        bc  = Ubar.boundaries[name]
        pbc = p.boundaries[name]
        for (i,f) ∈ enumerate(bc.faces)
            o = f.owner
            Ub = boundary_value(bc, i)
            pb = boundary_value(pbc, i)
            pn = (pb - p[o]) / norm(mesh.cells[o].x - f.x)
            ϕ[f.id] = dot(f.s, Ub) - ra[o] * pn
        end
    end

end

calculate_ϕ! (generic function with 1 method)

In [7]:
function SIMPLE(U, p, ν; fix_pressure=true)
    mesh = U.mesh
    α = 0.7
    β = 0.3

    ϕ = interpolate_U_to_faces(U)

    for iter = 0:50
    
        UOld, pOld = copy(U.values), copy(p.values)
    
        
        UEqn = div(ϕ,U) - Δ(ν,U)

        relax!(UEqn, α)
    
        solve!(UEqn + ∇(p))
    
        ra = 1 ./ Ac(UEqn);

        Ubar = VectorField(ra .* H(UEqn), U.mesh, U.boundaries);
    
        pEqn = Δ(ra, p) - ∇(Ubar);
        if fix_pressure
            pEqn.A[1,1] -= length(mesh.cells)
        end
        solve!(pEqn)
    
        p ← β*p + (1-β)*pOld
        U ← Ubar - ra .* ∇(p)
    
        calculate_ϕ!(ϕ, Ubar, p, ra)
        
        if rem(iter,5)==0
            nxny = length(mesh.cells)
            pRez = norm(pOld - p.values) / nxny
            URez = norm(UOld - U.values) / nxny
            println(iter, "\t", pRez, "\t", URez)
        end

    end
    
    return U,p
end

SIMPLE (generic function with 1 method)

In [8]:
function cavity_ns(ν, mesh)

    U,p = create_fields(mesh);
    U,p = SIMPLE(U, p, ν)
    return U,p
end

cavity_ns (generic function with 1 method)

In [ ]:
ν = 0.01;
msh25 = cartesian_mesh(25,25);
U,p = cavity_ns(ν, msh25);

In [ ]:
component(f::VectorField, c) = [ v[c] for v in f.values ]

In [ ]:
n=25
uu = reshape(component(U,1), (n,n));
vv = reshape(component(U,2), (n,n));
pp = reshape(p.values, (n,n));

In [ ]:
contourf(pp'); colorbar();
quiver(uu',vv');

## Jemnější síť a menší viskozita

In [ ]:
ν = 1.e-3;
msh50 = cartesian_mesh(50,50);

In [ ]:
U,p = cavity_ns(ν, msh50);

In [ ]:
n=50
uu = reshape(component(U,1), (n,n));
vv = reshape(component(U,2), (n,n));
pp = reshape(p.values, (n,n));

In [ ]:
contourf(pp'); colorbar();
quiver(uu',vv');

In [ ]:
# Ψ_x = v, Ψ_y = - u

function streamfunction(u, v, Δx=1, Δy=1)
    Ψ1 = zeros(u)

    Ψ1[1,1] = 0.0
    for i=2:size(u,1)
        Ψ1[i,1] = Ψ1[i-1,1] + Δx * (v[i,1]+v[i-1,1])/2
    end
    for j=2:size(u,2)
        Ψ1[:,j] = Ψ1[:,j-1] - Δy * (u[:,j]+u[:,j-1])/2
    end

    Ψ2 = zeros(Ψ1)
    
    Ψ2[1,1] = 0.0
    for j=2:size(u,2)
        Ψ2[1,j] = Ψ2[1,j-1] - Δy * (u[1,j,1]+u[1,j-1])/2
    end
    for i=2:size(u,1)
        Ψ2[i,:] = Ψ2[i-1,:] + Δx * (v[i,:]+v[i-1,:])/2
    end
    
    Ψ = (Ψ1 + Ψ2) / 2
    
    #Ψ[end,end] = (Ψ[end-1,end]+Ψ[end,end-1]) / 2
    return Ψ
end

In [ ]:
Ψ = streamfunction(uu,vv,1./25,1./25);

pos = linspace(0, maximum(Ψ), 10)
contour(Ψ', pos, colors="black");
neg = linspace(minimum(Ψ), 0, 5)
contourf(pp'); colorbar();
contour(Ψ', neg, colors="red");

## Vypočet na nestrukturované síti

In [ ]:
include("gmsh_mesh.jl")

In [ ]:
gmsh = gmsh_mesh("domain.msh");

In [ ]:
length(gmsh.cells)

In [ ]:
keys(gmsh.patches)

In [ ]:
U = VectorField(gmsh)
set_dirichlet_patch!(U, "INLET", Vec2d(1,0));
set_dirichlet_patch!(U, "TOP", Vec2d(0,0));
set_dirichlet_patch!(U, "BOTTOM", Vec2d(0,0));
set_dirichlet_patch!(U, "PROFILE", Vec2d(0,0));
set_neumann_patch!(U, "OUTLET", Vec2d(0,0));
U.values = [Vec2d(1,0) for c in gmsh.cells]

p = ScalarField(gmsh);
for name ∈ ["INLET", "PROFILE", "TOP", "BOTTOM"]
    set_neumann_patch!(p, name, 0.0)
end
set_dirichlet_patch!(p, "OUTLET", 0.0);

In [ ]:
U,p = SIMPLE(U, p, 1.e-2, fix_pressure=false);

In [ ]:
x = [c.x[1] for c in gmsh.cells];
y = [c.x[2] for c in gmsh.cells];
ux = [u[1] for u in U.values];
uy = [u[2] for u in U.values];

quiver(x,y,ux,uy);
axis("equal");